In [9]:
#If packages not installed
'''
%pip install numpy
%pip install ffmpeg-python
'''

'\n%pip install numpy\n%pip install ffmpeg-python\n'

In [1]:
import ffmpeg
import numpy as np

In [2]:
class Image:

    y = 0
    u = 0
    v = 0

    def __init__(self, r, g, b):
        self.r = r
        self.g = g
        self.b = b

    def rgb_to_yuv(self):
        self.y = (0.257 * self.r) + (0.504 * self.g) + (0.098 * self.b) + 16
        self.u = (-0.148 * self.r) - (0.291 * self.g) + (0.439 * self.b) + 128
        self.v = (0.439 * self.r) - (0.368 * self.g) - (0.071 * self.b) + 128
    
    def yuv_to_rgb(self):
        self.b = 1.164 * (self.y - 16) + 2.018 * (self.u - 128)
        self.g = 1.164 * (self.y - 16) - 0.813 * (self.v - 128) - 0.391 * (self.u - 128)
        self.r = 1.164 * (self.y - 16) + 1.596 * (self.v - 128) 
    
    def print_RGB_values(self):
        print("R:", self.r, "\n")
        print("G:", self.g, "\n")
        print("B:", self.b, "\n")

    def print_YUV_values(self):
        print("Y:", self.y, "\n")
        print("U:", self.u, "\n")
        print("V:", self.v, "\n")

In [3]:
#Exercise 2
i1 = Image(100, 50, 255)
i1.print_RGB_values()
i1.rgb_to_yuv()
i1.print_YUV_values()
i1.yuv_to_rgb()
i1.print_RGB_values()

R: 100 

G: 50 

B: 255 

Y: 91.89 

U: 210.59500000000003 

V: 135.395 

R: 100.13838000000001 

G: 50.02917999999999 

B: 255.01267000000004 



In [4]:

#Exercise 3
out=(
    ffmpeg
    .input('image.jpg')
    .filter("scale", 1080,1080)
    .output('out2.jpg')
    .run(capture_stdout=True)
)
# ffmpeg
# .input('image.jpg')
# .filter("scale", 368,5040).input('out.jpg')
#resize_image = image.filter("scale", 368,5040)
#resize_image.input('out.jpg')


In [5]:
#Exercise 4
#Code by Paul Panzer taken from (https://stackoverflow.com/questions/57366966/serpentine-scan-pattern-generator)
def zigzagiter(dims):
    N = len(dims)
    idx = N*[0]
    drc = N*[1]
    while True:
        yield (*idx,)
        for j in reversed(range(N)):
            if idx[j] + drc[j] not in (-1, dims[j]):
                idx[j] += drc[j]
                break
            drc[j] *= -1
        else:
            break
###

In [6]:
path = 'coltrane.jpg'
coltrane = ffmpeg.input(path)

probe = ffmpeg.probe(path)

image_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

width = int(image_info['width'])
height = int(image_info['height'])

out, err = (
    ffmpeg
    .input(path)
    .output('pipe:', format='rawvideo', pix_fmt='rgb24')
    .run(capture_stdout=True)
)

coltrane_array = (
    np
    .frombuffer(out, np.uint8)
    .reshape([-1, height, width, 3])
)

In [16]:
image_squeezed = np.squeeze(coltrane_array)

r = image_squeezed[:, :, 0]
g = image_squeezed[:, :, 1]
b = image_squeezed[:, :, 2]

In [ ]:
# Exercise 5.1:
path = 'superbowl.jpg'
coltrane = ffmpeg.input(path)

probe = ffmpeg.probe(path)

image_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')

width = int(image_info['width'])
height = int(image_info['height'])


ffmpeg.output(
    coltrane.filter('hue', s=0),
    'output_image_bw.jpg'
).run()



(None, None)

In [ ]:
#We assumed that the chain is given in an array. like: [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1]

def codif_bits(bits_chain):
    length = len(bits_chain)
    encoding = ""
    i = 0
    while i < length:
        count = 1
        while i + 1 < length and bits_chain[i] == bits_chain[i + 1]:
            count += 1
            i += 1

        encoding += str(bits_chain[i]) + str(count)
        i += 1
    
    return encoding



In [4]:
bits = [1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1] 
encoded_bits = codif_bits(bits) 
print(encoded_bits)

1403110212
